# AI Services Example Notebook

This notebook demonstrates how to use the `ai_services.py` module from the WorkoutBuddy ML backend.

## Features Covered:
1. **Personalized Challenge Generation** - AI-powered daily workout challenges
2. **Community Compatibility Matching** - Find compatible workout partners
3. **Personalized Encouragement** - Generate motivational messages

## Prerequisites:
- Set up the `ANTHROPIC_API_KEY` in `.envrc` file
- Install required dependencies from `requirements.txt`
- Have access to the database models

In [ ]:
# Import required modules
import sys
import os
import asyncio
from datetime import datetime, timedelta
from typing import Dict, List

# Add the ml_backend path to sys.path
sys.path.append('ml_backend')

# Load environment variables from .envrc file
def load_envrc():
    """Load environment variables from .envrc file"""
    try:
        with open('.envrc', 'r') as f:
            for line in f:
                line = line.strip()
                if line and not line.startswith('#') and '=' in line:
                    key, value = line.split('export ', 1)[1].split('=', 1)
                    os.environ[key] = value
        print("✅ Environment variables loaded from .envrc")
    except FileNotFoundError:
        print("⚠️  .envrc file not found. Please create one with ANTHROPIC_API_KEY")
    except Exception as e:
        print(f"❌ Error loading .envrc: {e}")

# Load environment variables
load_envrc()

print("Environment setup complete!")
print(f"Python version: {sys.version}")
print(f"Working directory: {os.getcwd()}")
print(f"API Key configured: {'✅' if os.getenv('ANTHROPIC_API_KEY') else '❌'}")

In [ ]:
# Import the AI service
try:
    from app.ai_services import ai_service, ChallengeResponse, CommunityMatchResponse, EncouragementResponse
    from app.core import models
    print("✅ AI Services imported successfully!")
    print(f"AI Service enabled: {ai_service.enabled}")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("Make sure you're running from the correct directory and have installed dependencies")

## 1. Setup Mock Data

Since we're running this notebook independently, we'll create mock user objects that match the database model structure.

In [ ]:
# Create mock user class for testing
class MockUser:
    def __init__(self, user_id, goal_type="cardio", activity_level="moderate", 
                 year_of_birth=1990, first_name="John", last_name="Doe"):
        self.id = user_id
        self.goal_type = goal_type
        self.activity_level = activity_level
        self.year_of_birth = year_of_birth
        self.first_name = first_name
        self.last_name = last_name
        self.motivation_style = "encouraging"
        self.community_engagement_score = 0.7

class MockGoal:
    def __init__(self, goal_id, title, user_id):
        self.id = goal_id
        self.title = title
        self.owner_id = user_id

# Create sample users
user1 = MockUser(1, "cardio", "moderate", 1985, "Alice", "Johnson")
user2 = MockUser(2, "strength", "high", 1990, "Bob", "Smith")
user3 = MockUser(3, "flexibility", "low", 1995, "Carol", "Davis")
user4 = MockUser(4, "cardio", "moderate", 1988, "David", "Wilson")

# Create sample goals
goals_user1 = [MockGoal(1, "Run 5K in under 25 minutes", 1), MockGoal(2, "Improve cardiovascular health", 1)]
goals_user2 = [MockGoal(3, "Bench press bodyweight", 2), MockGoal(4, "Build muscle mass", 2)]
goals_user3 = [MockGoal(5, "Touch toes without bending knees", 3)]
goals_user4 = [MockGoal(6, "Complete a marathon", 4)]

print("Mock users and goals created successfully!")
print(f"User 1: {user1.first_name} {user1.last_name} - Goal: {user1.goal_type}, Activity: {user1.activity_level}")
print(f"User 2: {user2.first_name} {user2.last_name} - Goal: {user2.goal_type}, Activity: {user2.activity_level}")
print(f"User 3: {user3.first_name} {user3.last_name} - Goal: {user3.goal_type}, Activity: {user3.activity_level}")
print(f"User 4: {user4.first_name} {user4.last_name} - Goal: {user4.goal_type}, Activity: {user4.activity_level}")

## 2. Personalized Challenge Generation

Generate AI-powered personalized daily challenges based on user profile and history.

In [ ]:
# Example user history and preferences
user_history = {
    "recent_completion_rate": "85% - Very consistent",
    "recent_challenges": [
        "20-minute HIIT workout",
        "Morning jog routine",
        "Bodyweight circuit"
    ]
}

user_preferences = {
    "equipment": "Dumbbells",
    "time_minutes": 20,
    "preferred_intensity": "moderate"
}

# Generate personalized challenge
async def demo_challenge_generation():
    print("\n=== PERSONALIZED CHALLENGE GENERATION ===")
    print(f"Generating challenge for: {user1.first_name} {user1.last_name}")
    print(f"Goal Type: {user1.goal_type}")
    print(f"Activity Level: {user1.activity_level}")
    print(f"Age: {datetime.now().year - user1.year_of_birth}")
    
    challenge = await ai_service.generate_personalized_challenge(
        user=user1,
        user_history=user_history,
        preferences=user_preferences
    )
    
    print("\n📋 Generated Challenge:")
    print(f"Title: {challenge.title}")
    print(f"Description: {challenge.description}")
    print(f"Duration: {challenge.duration}")
    print(f"Difficulty: {challenge.difficulty}/5")
    print(f"Equipment Needed: {challenge.equipment_needed}")
    print(f"Motivation: {challenge.motivation_message}")
    print(f"AI Generated: {challenge.ai_generated}")
    
    return challenge

# Run the challenge generation
challenge_result = await demo_challenge_generation()

In [ ]:
async def test_different_user_challenges():
    users_to_test = [user2, user3]  # Strength and flexibility users
    
    for user in users_to_test:
        print(f"\n\n=== CHALLENGE FOR {user.first_name.upper()} ===")
        print(f"Goal: {user.goal_type} | Activity Level: {user.activity_level}")
        
        # Adjust history based on user type
        adapted_history = {
            "recent_completion_rate": "70% - Good progress",
            "recent_challenges": [
                f"{user.goal_type.title()} focused workout",
                "Beginner routine"
            ]
        }
        
        challenge = await ai_service.generate_personalized_challenge(
            user=user,
            user_history=adapted_history,
            preferences={"time_minutes": 15}
        )
        
        print(f"📋 {challenge.title}")
        print(f"📝 {challenge.description}")
        print(f"⏱️ Duration: {challenge.duration}")
        print(f"💪 Motivation: {challenge.motivation_message}")

await test_different_user_challenges()

## 3. Community Compatibility Matching

Demonstrate AI-powered matching between users for accountability partnerships.

In [ ]:
async def demo_community_matching():
    print("\n\n=== COMMUNITY COMPATIBILITY MATCHING ===")
    print(f"Finding matches for: {user1.first_name} {user1.last_name}")
    print(f"Looking for accountability partners...")
    
    # Potential matches (excluding the user themselves)
    potential_matches = [user2, user3, user4]
    
    # Goals for each user
    match_goals = {
        user2.id: goals_user2,
        user3.id: goals_user3,
        user4.id: goals_user4
    }
    
    matches = await ai_service.analyze_community_compatibility(
        user=user1,
        potential_matches=potential_matches,
        user_goals=goals_user1,
        match_goals=match_goals
    )
    
    print(f"\n🤝 Found {len(matches)} potential matches:")
    print("=" * 50)
    
    for i, match in enumerate(matches, 1):
        print(f"\n#{i} Match: {match.name}")
        print(f"🎯 Compatibility Score: {match.compatibility_score:.1%}")
        print(f"💡 Why they're a good match:")
        for reason in match.match_reasons:
            print(f"   • {reason}")
        print(f"🏃 Shared Interests: {', '.join(match.shared_interests)}")
    
    return matches

matches_result = await demo_community_matching()

## 4. Personalized Encouragement Generation

Generate contextual encouragement messages based on user progress and sentiment.

In [ ]:
async def demo_encouragement_generation():
    print("\n\n=== PERSONALIZED ENCOURAGEMENT GENERATION ===")
    
    # Different user contexts to test
    test_scenarios = [
        {
            "name": "High Performer",
            "context": {
                "recent_checkin": "Completed 5K run in 24:30 - new personal record!",
                "progress_trend": "Improving consistently",
                "engagement_level": "High",
                "days_since_last": 1,
                "recent_challenges_completed": 6
            }
        },
        {
            "name": "Struggling User",
            "context": {
                "recent_checkin": "Missed workouts this week, feeling unmotivated",
                "progress_trend": "Declining",
                "engagement_level": "Low",
                "days_since_last": 7,
                "recent_challenges_completed": 1
            }
        },
        {
            "name": "Comeback User",
            "context": {
                "recent_checkin": "Back to working out after a break",
                "progress_trend": "Stable",
                "engagement_level": "Medium",
                "days_since_last": 3,
                "recent_challenges_completed": 2
            }
        }
    ]
    
    for scenario in test_scenarios:
        print(f"\n📱 Scenario: {scenario['name']}")
        print(f"Context: {scenario['context']['recent_checkin']}")
        
        encouragement = await ai_service.generate_encouragement(
            user=user1,
            context=scenario['context']
        )
        
        print(f"\n💬 Generated Encouragement:")
        print(f"Message: \"{encouragement.message}\"")
        print(f"Tone: {encouragement.tone}")
        print(f"Personalized: {encouragement.personalized}")
        print(f"Suggestions:")
        for suggestion in encouragement.suggestions:
            print(f"   • {suggestion}")
        print("-" * 50)

await demo_encouragement_generation()

## 5. Fallback Behavior Testing

Test how the service behaves when AI is unavailable (without API key).

In [ ]:
async def demo_fallback_behavior():
    print("\n\n=== FALLBACK BEHAVIOR TESTING ===")
    print("Testing AI service without API key (fallback mode)")
    
    # Create a service instance without API key
    from app.ai_services import AIService
    
    # Temporarily remove API key
    original_key = os.environ.get('ANTHROPIC_API_KEY')
    if 'ANTHROPIC_API_KEY' in os.environ:
        del os.environ['ANTHROPIC_API_KEY']
    
    fallback_service = AIService()
    
    print(f"AI Service enabled: {fallback_service.enabled}")
    
    # Test fallback challenge generation
    print("\n📋 Fallback Challenge Generation:")
    fallback_challenge = await fallback_service.generate_personalized_challenge(
        user=user2,  # strength user
        user_history=user_history
    )
    
    print(f"Title: {fallback_challenge.title}")
    print(f"Description: {fallback_challenge.description}")
    print(f"AI Generated: {fallback_challenge.ai_generated}")
    
    # Test fallback encouragement
    print("\n💬 Fallback Encouragement:")
    fallback_encouragement = await fallback_service.generate_encouragement(
        user=user1,
        context={"progress_trend": "stable"}
    )
    
    print(f"Message: \"{fallback_encouragement.message}\"")
    print(f"Personalized: {fallback_encouragement.personalized}")
    
    # Restore API key
    if original_key:
        os.environ['ANTHROPIC_API_KEY'] = original_key
    
    print("\n✅ Fallback testing completed!")

await demo_fallback_behavior()

## 6. Usage Instructions

### Environment Setup
```bash
# Make sure .envrc file contains your API key:
# export ANTHROPIC_API_KEY="your-api-key-here"

# If using direnv (recommended):
direnv allow

# Or manually source the file:
source .envrc

# Install dependencies
pip install -r ml_backend/requirements.txt

# Start Jupyter
jupyter notebook
```

### Production Considerations
1. **Rate Limiting**: Implement rate limiting to avoid API quota issues
2. **Caching**: Cache similar requests to reduce costs
3. **Error Handling**: Always have fallback responses ready
4. **Monitoring**: Track API usage and costs
5. **User Consent**: Ensure users consent to AI-generated content
6. **Security**: Never commit API keys to version control

### Integration Example
```python
from app.ai_services import ai_service

@app.post("/generate-challenge")
async def generate_challenge(user_id: int, db: Session = Depends(get_db)):
    user = db.query(User).filter(User.id == user_id).first()
    challenge = await ai_service.generate_personalized_challenge(
        user=user,
        user_history=get_user_history(user_id),
        preferences=get_user_preferences(user_id)
    )
    return challenge
```

In [ ]:
print("\n\n🎉 AI Services Demo Complete!")
print("\nKey Features Demonstrated:")
print("✅ Personalized Challenge Generation")
print("✅ Community Compatibility Matching")
print("✅ Personalized Encouragement")
print("✅ Fallback Behavior")
print("\nThe AI service is ready for integration into your WorkoutBuddy application!")